In [1]:
import openai

/Users/htang/Documents/amazon-bedrock-rag-workshop/bedragenv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [31]:
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [34]:
import fitz  # PyMuPDF
import os


def read_pdf2Text(pdf_directory, pdf_filenames):
    extracted_texts = {}
    for filename in pdf_filenames:
        pdf_path = os.path.join(pdf_directory, filename)
        with fitz.open(pdf_path) as pdf:
            text = ""
            for page in pdf:
                text += page.get_text()
            extracted_texts[filename] = text
    return extracted_texts

# Define the directory containing the PDFs
pdf_directory = './data'

# Read all file names in the pdf_directory that end with '.pdf'
pdf_filenames = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]


extracted_texts=read_pdf2Text(pdf_directory, pdf_filenames)
# Return the number of extracted texts to confirm extraction
len(extracted_texts)

pdf_filename=['search_resume.pdf']
pdf_directory='.'
new_resume_content =read_pdf2Text(pdf_directory, pdf_filename)
len(new_resume_content)

15

'Machine Learning Engineer Resume\nContact Information\nName: Jordan Smith\nAddress: 500 Tech Avenue, Techville, USA\nPhone: (555) 500-5000\nEmail: jordan.smith@example.com\nLinkedIn: linkedin.com/in/jordansmith\nGitHub: github.com/jordansmith\nMachine Learning Engineer Resume\nProfessional Summary\nInnovative and results-driven Machine Learning Engineer with over three years of experience in\ndeveloping high-performance predictive models and data-driven solutions. Proficient in harnessing\nmachine learning to solve complex problems and improve business outcomes. Seeking to apply\nexpertise in data science and software engineering at an industry-leading tech firm.\nMachine Learning Engineer Resume\nEducation\nM.S. in Computer Science, Specialization in Machine Learning\nTech University, May 2020\nThesis: "Scalable Machine Learning Algorithms for Big Data Environments"\nRelevant Coursework: Machine Learning, Predictive Analytics, Data Structures and Algorithms,\nCloud Computing\nMachine

In [71]:
import openai

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']


In [73]:
import numpy as np
import json
import pandas as pd
extracted_texts_embeddings={}
for key, text in extracted_texts.items():
    extracted_texts_embeddings[key] = get_embedding(text, model="text-embedding-ada-002")
# extracted_texts_embeddings

In [81]:
resume_embeddings = get_embedding(list(new_resume_content.values())[0], model="text-embedding-ada-002")

In [84]:


# Calculate similarity (cosine similarity) between the resume and each PDF's embedding
def calculate_similarity(embedding1, embedding2):
    # Define a function for cosine similarity
    dot_product = sum(a*b for a, b in zip(embedding1, embedding2))
    norm_a = sum(a*a for a in embedding1) ** 0.5
    norm_b = sum(b*b for b in embedding2) ** 0.5
    return dot_product / (norm_a * norm_b)

# Calculate similarities for all PDFs
similarities = [calculate_similarity(resume_embeddings, pdf_emb) for pdf_emb in extracted_texts_embeddings.values()]


# Create a DataFrame to display these similarities
pdf_similarity_df = pd.DataFrame(zip(pdf_filenames, similarities), columns=['PDF File', 'Similarity Score'])

# Sort the DataFrame by the similarity score in descending order
sorted_pdf_similarity_df = pdf_similarity_df.sort_values(by='Similarity Score', ascending=False)

# Display the sorted DataFrame to find the top matches
print(sorted_pdf_similarity_df.head(2))


                                  PDF File  Similarity Score
13  Machine_Learning_Engineer_Resume_5.pdf          0.934206
14  Machine_Learning_Engineer_Resume_4.pdf          0.932344
